In [1]:
import pandas as pd
import numpy as np
import os
import re
import glob
import dask.dataframe as dd
import torch
import chess

In [2]:
parquets = glob.glob('../data/processed/*.parquet')

In [3]:
# one_million_games = pd.concat([pd.read_parquet(parquet) for parquet in parquets])

In [4]:
def generate_full_uci_move_vocabulary() -> tuple[dict,dict]:

    """
    Generates a set of all the posible movements in a chess board of 64 squares, 
    create two dictionaries which will represent the board move in uci format and their respective idx value and viceversa
    
    Returns
    -------
        uci_to_idx : dict
                     All the possible uci moves in a chess board, uci format as keys and idx as values
        idx_to_uci : dict 
                     All the possible uci moves in a chess board, uci format as keys and idx as values
    
    """
    move_set = set()
    
    for from_sq in chess.SQUARES:
        for to_sq in chess.SQUARES:
            if from_sq == to_sq:
                continue

            move = chess.Move(from_sq, to_sq)
            move_set.add(move.uci())
            
            from_rank = chess.square_rank(from_sq) # Get the row in which the piece is coming from
            to_rank = chess.square_rank(to_sq) # Get the row in which will be moved the piece
            # if to_rank in [0, 7] and from_rank in [1,6]:  # posibles promociones
            if (from_rank == 1 and to_rank == 0) or (from_rank == 6 and to_rank ==7):
                
                for promo in [chess.QUEEN, chess.ROOK, chess.BISHOP, chess.KNIGHT]:
                    move_set.add(chess.Move(from_sq, to_sq, promotion=promo).uci())
                    
    move_list = sorted(move_set)
    uci_to_idx = {uci: idx for idx, uci in enumerate(move_list)}
    idx_to_uci = {idx: uci for uci, idx in uci_to_idx.items()}
    return uci_to_idx, idx_to_uci

def fen_to_tensor(fen:str) -> torch.Tensor:
    """
    Converts a FEN position into a torch tensor of shape (12,8,8),
    12 matrix of 8x8 positions, in which each type of piece eaither PNBRQK or pnbrqk,
    will ocupate a place in the matrix, each matrix for each set of piece representation.

    Parameters
    ----------
    fen : str
          The notation FEN to convert into numerical values
    Returns
    -------
    board_tensor : torch.Tensor
                   The representation of FEN notation in 12 matrix of 8x8

    """

    
    board = chess.Board(fen)
    
    piece_to_index = {piece:idx for idx,piece in enumerate('PNBRQKpnbrqk')} # represents the piece and index of each value of the str

    #TODO: add extra ccanals to indicate if there is castling available 4 canals, passant square, halfmove clock
    
    board_tensor = torch.zeros((12,8,8),dtype=torch.float32)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            idx = piece_to_index[piece.symbol()]
            row = 7 - (square // 8)
            col = square %8
            board_tensor[idx,row,col] = 1.0
    return board_tensor


def get_legal_moves_vocab(fen:str) -> tuple[dict[str,int],dict[int,str]]:
    """
    Generates a set of legal posible moves for a given position 

    IMPORTANT ---> All the dict generated are LOCAL and could not match with the global dict --> generate_full_uci_move_vocabulary()

    Parameters
    ----------
        fen: FEN notation of the current position
    Returns
    -------
        uci_to_idx: Dict {uci_move : idx}
        idc_to_uci: Dict {idx : uci_move}
    """

    board = chess.Board(fen)
    legal_moves = list(board.legal_moves)
    
    legal_moves_sorted = sorted(legal_moves, key=lambda m: m.uci())

    uci_to_idx = {move.uci():  idx for idx, move in enumerate(legal_moves_sorted)}
    idx_to_uci = {idx: move.uci() for idx,move in enumerate(legal_moves_sorted)}
    return uci_to_idx, idx_to_uci

 ## POSIBLEMENTE DESCARTADO, MEJORA ALTERNATIVA CON FUNCION  --> get_legal_moves_vocab enfoque "SPARSE"
# def get_legal_mask(board: chess.Board, uci_to_index: dict) -> torch.Tensor:
#     mask = torch.zeros(len(uci_to_index), dtype=torch.float32)
#     for move in board.legal_moves:
#         uci = move.uci()
#         if uci in uci_to_index:
#             mask[uci_to_index[uci]] = 1.0
#     return mask  # Shape: (n_moves,)

    


    move_list = sorted(move_set)
    uci_to_index = {uci: idx for idx, uci in enumerate(move_list)}
    index_to_uci = {idx: uci for uci, idx in uci_to_index.items()}
    return uci_to_index, index_to_uci
# Globales cargados una vez al inicio

def move_to_index(uci_move: str) -> int:
    return uci_to_index.get(uci_move, -1)  # -1 si no está

def index_to_move(idx: int) -> str:
    return index_to_uci.get(idx, "0000")  # dummy por si acaso

        

In [34]:
class ChessSequenceDataset(torch.utils.data.Dataset):

    def __init__(self,df,uci_to_idx):

        
        self.games = []
        self.uci_to_idx = uci_to_idx
        grouped = df.groupby('game_id')

        for game_id, group in grouped:
            group_sorted = group.sort_values(by='ply',ascending=True)#group.sort_values(by='pyl',ascending=True)
            sequence = []

            for _,row in group_sorted.iterrows():
                fen = row['fen']
                uci = row['move_uci']

                move_idx = uci_to_idx.get(uci,-1)
                if move_idx ==-1:
                    continue
                try:
                    fen_tensor = fen_to_tensor(fen)
                except Exception as e:
                    print(f'Error in FEN {fen} --->{e}')
                    continue
                sequence.append((fen_tensor,move_idx))
            if len(sequence)>0:
                self.games.append(sequence)
                
    def __len__(self):
        return len(self.games)
    def __getitem__(self,idx):
        return self.games[idx]
        
    

In [35]:
class ChessLSTMPolicyNet(torch.nn.Module):
    def __init__(self,input_channels=12,lstm_hidden_size=512,lstm_layers=1,output_dim=4544):
        super().__init__()

        
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(input_channels,64,kernel_size=3,padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64,128,kernel_size=3,padding=1),
            torch.nn.ReLU()
        )
        self.flattened_size = 128*8*8
        self.lstm = torch.nn.LSTM(
            input_size= self.flattened_size,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_layers,
            batch_first=True
        )


        self.fc = torch.nn.Linear(lstm_hidden_size,output_dim)


        def forward(self,x):
                B, T, C, H, W = x.shape
                x = x.view(B * T, C, H, W)           # (B*T, C, 8, 8)
                x = self.conv(x)                     # (B*T, 128, 8, 8)
                x = x.view(B, T, -1)                 # (B, T, 8192)
        
                lstm_out, _ = self.lstm(x)           # (B, T, hidden)
                logits = self.fc(lstm_out)           # (B, T, output_dim)
                return logits

In [36]:
def train_lstm(model, dataloader, criterion, optimizer, epochs=5):
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        total_correct = 0
        total_moves = 0

        for batch in dataloader:
            # batch: list of 1 element (sequence of (fen_tensor, move_idx))
            sequence = batch[0]

            inputs = torch.stack([x[0] for x in sequence])  # (T, C, 8, 8)
            targets = torch.tensor([x[1] for x in sequence])  # (T,)

            # Reshape to (B, T, C, 8, 8)
            inputs = inputs.unsqueeze(0).to(device)
            targets = targets.unsqueeze(0).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)  # (B, T, output_dim)

            # Aplanar para CrossEntropy
            logits = outputs.view(-1, outputs.size(-1))     # (T, output_dim)
            target_flat = targets.view(-1)                  # (T,)

            loss = criterion(logits, target_flat)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            total_correct += (preds == target_flat).sum().item()
            total_moves += target_flat.size(0)

        acc = total_correct / total_moves
        print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}, Accuracy = {acc:.4f}")


In [37]:
df = pd.read_parquet(parquets[0])

In [38]:
game_ids = list(df['game_id'].unique())

In [39]:
np.random.choice?

Signature: np.random.choice(a, size=None, replace=True, p=None)
Docstring:
choice(a, size=None, replace=True, p=None)

Generates a random sample from a given 1-D array

.. versionadded:: 1.7.0

.. note::
    New code should use the `~numpy.random.Generator.choice`
    method of a `~numpy.random.Generator` instance instead;
    please see the :ref:`random-quick-start`.

.. warning::
    This function uses the C-long dtype, which is 32bit on windows
    and otherwise 64bit on 64bit platforms (and 32bit on 32bit ones).
    Since NumPy 2.0, NumPy's default integer is 32bit on 32bit platforms
    and 64bit on 64bit platforms.


Parameters
----------
a : 1-D array-like or int
    If an ndarray, a random sample is generated from its elements.
    If an int, the random sample is generated as if it were ``np.arange(a)``
size : int or tuple of ints, optional
    Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
    ``m * n * k`` samples are drawn.  Default is None, in which case a


In [40]:
game_samples_id = np.random.choice(game_ids,size=10_000,replace=False)

In [17]:
game_samples_df = df.loc[df['game_id'].isin(game_samples_id)].copy()

In [22]:
uci_to_idx,idx_to_uci = generate_full_uci_move_vocabulary()

In [23]:
game_samples_df.head()

,fen,move_uci,move_san,player,result,game_id,full_move,ply
2007,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,h2h4,h4,white,-1,26,1,0
2008,rnbqkbnr/pppppppp/8/8/7P/8/PPPPPPP1/RNBQKBNR b...,e7e5,e5,black,-1,26,1,1
2009,rnbqkbnr/pppp1ppp/8/4p3/7P/8/PPPPPPP1/RNBQKBNR...,d2d4,d4,white,-1,26,2,2
2010,rnbqkbnr/pppp1ppp/8/4p3/3P3P/8/PPP1PPP1/RNBQKB...,f7f6,f6,black,-1,26,2,3
2011,rnbqkbnr/pppp2pp/5p2/4p3/3P3P/8/PPP1PPP1/RNBQK...,d4d5,d5,white,-1,26,3,4


In [ ]:
torch.utils

In [42]:
# Crear dataset y dataloader (batch_size = 1 por ahora)
dataset = ChessSequenceDataset(game_samples_df, uci_to_idx)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [43]:
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [44]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ChessLSTMPolicyNet(
    input_channels=12,
    lstm_hidden_size=512,
    output_dim=len(uci_to_idx)
).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [46]:
train_lstm(model, loader, criterion, optimizer, epochs=5)


RuntimeError: stack expects each tensor to be equal size, but got [12, 8, 8] at entry 0 and [] at entry 1

In [52]:
for _,data in enumerate(loader):
    if _>= 1:
        break
        for position in data:
            print(position)
    print(data)

[[tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 1., 1., 1., 1.],
          [0., 0., 0., 0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0., 0., 1., 0.]],

         [[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0.,